Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [49]:
# Logistic
num_labels = 10
pixelCount = 28 * 28
train_subset = 10000
num_steps = 801
# t paramater for our L2 Loss
t = 0.25

graph = tf.Graph()
with graph.as_default():
    # Configure Constants
    tf_TrainSet = tf.constant(train_dataset[:train_subset, :])
    tf_TrainLabels = tf.constant(train_labels[:train_subset, :])
    tf_TestSet = tf.constant(test_dataset)
    tf_ValidSet = tf.constant(valid_dataset)
    
    # Setup the logistic regression variables
    weights = tf.Variable(
        tf.truncated_normal([pixelCount, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    log = tf.matmul(tf_TrainSet, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(log, tf_TrainLabels))
    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
    # Add the regularization term to the loss.
    #c = 5e-4
    c = 0.005
    loss += c * regularizers
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(log)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_ValidSet, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_TestSet, weights) + biases)
    
with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 33.023075
Training accuracy: 9.3%
Validation accuracy: 12.6%
Test accuracy: 12.7%
Loss at step 100: 10.524026
Training accuracy: 73.4%
Validation accuracy: 71.2%
Test accuracy: 78.6%
Loss at step 200: 6.227783
Training accuracy: 77.4%
Validation accuracy: 74.6%
Test accuracy: 82.0%
Loss at step 300: 3.817248
Training accuracy: 80.0%
Validation accuracy: 76.6%
Test accuracy: 83.6%
Loss at step 400: 2.446922
Training accuracy: 82.0%
Validation accuracy: 78.4%
Test accuracy: 85.3%
Loss at step 500: 1.667974
Training accuracy: 83.2%
Validation accuracy: 79.7%
Test accuracy: 86.6%
Loss at step 600: 1.224073
Training accuracy: 84.2%
Validation accuracy: 80.7%
Test accuracy: 87.5%
Loss at step 700: 0.969800
Training accuracy: 84.8%
Validation accuracy: 81.2%
Test accuracy: 88.0%
Loss at step 800: 0.823316
Training accuracy: 85.1%
Validation accuracy: 81.7%
Test accuracy: 88.4%


In [60]:
# Logistic
num_labels = 10
pixelCount = 28 * 28
train_subset = 10000
num_steps = 801
hNodes = 1024
# t paramater for our L2 Loss
t = 0.25

graph = tf.Graph()
with graph.as_default():
    # Configure Constants
    tf_TrainSet = tf.constant(train_dataset[:train_subset, :])
    tf_TrainLabels = tf.constant(train_labels[:train_subset, :])
    tf_TestSet = tf.constant(test_dataset)
    tf_ValidSet = tf.constant(valid_dataset)
    
    # Setup the logistic regression variables
    weights1 = tf.Variable(
        tf.truncated_normal([pixelCount, hNodes]))
    biases1 = tf.Variable(tf.zeros([hNodes]))
    weights2 = tf.Variable(
        tf.truncated_normal([hNodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    relu = tf.nn.relu(tf.matmul(tf_TrainSet, weights1) + biases1)
    logit = tf.matmul(relu, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logit, tf_TrainLabels))
    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1)
                   + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    # Add the regularization term to the loss.
    c = 5e-4
    #c = 0.005
    loss += c * regularizers
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logit)
    valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_ValidSet, weights1) + biases1)
            , weights2) + biases2)
    
    test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_TestSet, weights1) + biases1)
            , weights2) + biases2)

    
with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 443.445190
Training accuracy: 8.6%
Validation accuracy: 40.7%
Test accuracy: 44.1%
Loss at step 100: 151.718140
Training accuracy: 92.0%
Validation accuracy: 79.1%
Test accuracy: 86.7%
Loss at step 200: 141.414841
Training accuracy: 99.5%
Validation accuracy: 79.1%
Test accuracy: 86.4%
Loss at step 300: 134.497650
Training accuracy: 99.7%
Validation accuracy: 79.2%
Test accuracy: 86.8%
Loss at step 400: 127.926186
Training accuracy: 99.8%
Validation accuracy: 79.2%
Test accuracy: 86.9%
Loss at step 500: 121.682175
Training accuracy: 99.9%
Validation accuracy: 79.2%
Test accuracy: 86.9%
Loss at step 600: 115.733208
Training accuracy: 99.8%
Validation accuracy: 78.1%
Test accuracy: 85.3%
Loss at step 700: 110.094391
Training accuracy: 99.9%
Validation accuracy: 79.2%
Test accuracy: 87.0%
Loss at step 800: 104.716179
Training accuracy: 99.9%
Validation accuracy: 79.2%
Test accuracy: 86.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
